In [1]:
# вопрос - ответы на задания 3-6 нужны только для тех записей, где дан fuelLevel?

In [2]:
import pandas as pd
import numpy as np

In [3]:
DATA_PATH = 'data/vehicle_dataset_public/'
DATA_PROC_PATH = 'data/vehicle_proccessed/'

TAGS_FEATURES = ['fuelLevel', 'ingection', 'speedAndHeight', 'tachometer']
TAGS_TARGET = ['refueling2']
TAGS = TAGS_FEATURES + TAGS_TARGET

TIME_COLUMNS = ['DTIME', 'STARTDATE', 'ENDDATE']
FLOAT_COLUMNS = ['BEVALUE', 'SPEED', 'HEIGHT']

VEHICLE_IDS = [1, 3, 5, 19, 28]

In [9]:
from functools import reduce

# Словарь, состоящий из dataframe'ов для каждого признака
def dict_df_vehicle_i(i):
    dict_df_vehicle = {}
    for tag in TAGS:
        df = pd.read_csv(DATA_PATH +
                         f'vehicle{i}_{tag}_public.csv', sep=';')
        for col in FLOAT_COLUMNS:
            if col in df.columns and df[col].dtype == 'object':
                df[col] = df[col].str.replace(',','.').astype(float)
        df.rename(columns={'BEVALUE': tag}, inplace=True)

        for time_col in TIME_COLUMNS:
            if time_col in df.columns:
                df[time_col] = pd.to_datetime(df[time_col])

        dict_df_vehicle[tag] = df

    return dict_df_vehicle

# Объединение всех "исходных" признаков из словаря dataframe'ов
# в один dataframe
def df_summary(dict_df, merge_how='outer'):
    df_to_merge = [dict_df[name] for name in TAGS_FEATURES]

    df_res = reduce(lambda left, right: pd.merge(left, right, how=merge_how, on='DTIME'),
                    df_to_merge)
    df_res.columns = df_res.columns.str.lower()

    return df_res

In [24]:
def timeDeltaToMinutes(timeDelta):
    return np.dot(timeDelta.dt.components, [24*60, 60, 1, 1/60, 1/(60**2), 1/(60**3), 1/(60**4)])

In [33]:
# считает скорость измерения топлива по записям времени и уровня топлива
def compute_LPM(timeSeries, fuelLevelSeries):
    shiftedDTIME = np.full(len(timeSeries), np.nan)
    shiftedDTIME[1:] = timeSeries.values[:-1]
    shiftedDTIME = pd.Series(pd.to_datetime(shiftedDTIME))
    
    shiftedFuelLevel = np.full(len(timeSeries), np.nan)
    shiftedFuelLevel[1:] = fuelLevelSeries.values[:-1]
    shiftedDTIME = pd.Series(shiftedDTIME)

    timeShift = timeSeries - shiftedDTIME
    timeShiftMinutes = timeDeltaToMinutes(timeShift)

    fuelLevelShift = fuelLevelSeries - shiftedFuelLevel
    
    return fuelLevelShift / timeShiftMinutes

In [147]:
vehiclesFeatures = []

for vehicle_id in VEHICLE_IDS:
    dict_df = dict_df_vehicle_i(vehicle_id)
    
    df_sum = df_summary(dict_df, merge_how='inner')
    
    df_sum['LPM'] = compute_LPM(df_sum['dtime'], df_sum['fuellevel'])
    
    def mean_decrease_lpm(x):
        x = x[1:] # первая запись за день - изменение с предыдущего дня
        x = x[x<0] # только уменьшение
        return np.mean(x)
    
    # В одном жне может быть несколько последовательностей измерений,
    # наверное, нужно это тоже будет как то учесть. Пока так.
    each_day_mean_lpm = df_sum.groupby(df_sum['dtime'].dt.date)['LPM'].apply(mean_decrease_lpm)
    
    # MedianDecLPM - медиана средней скорости уменьшения топлива за день ._.
    
    features = {
        'MaxFuelLevel': df_sum['fuellevel'].max(), 
        'MeanFuelLevel': df_sum['fuellevel'].mean(),
        'MedianFuelLevel': df_sum['fuellevel'].median(),
        'MaxSpeed': df_sum['speed'].max(), 
        'MeanSpeed': df_sum['speed'].mean(),
        'MedianSpeed': df_sum['speed'].median(),
        'SpeedStd': df_sum['speed'].std(),
        'MeanTachometer': df_sum['tachometer'].mean(),
        'MedianTachometer': df_sum['tachometer'].median(),
        'MeanIngection': df_sum['ingection'].mean(),
        'MedianDecLPM': each_day_mean_lpm.median(),
    }
    
    vehicleWiseFeatures = pd.Series(features)
    
    vehiclesFeatures.append(vehicleWiseFeatures)


In [149]:
X = pd.DataFrame(vehiclesFeatures, index=VEHICLE_IDS)
X

,MaxFuelLevel,MeanFuelLevel,MedianFuelLevel,MaxSpeed,MeanSpeed,MedianSpeed,SpeedStd,MeanTachometer,MedianTachometer,MeanIngection,MedianDecLPM
1,60.0,38.865364,40.00,128.0,36.009615,24.0,37.117291,1807.216478,1568.0,0.984815,-1.053105
3,279.8,180.684452,187.85,105.0,15.581290,0.0,24.016025,1032.753226,950.0,0.999032,-0.554609
5,73.5,40.347142,41.10,131.0,44.640783,46.0,37.745430,1572.068366,1686.0,0.976389,-1.221998
19,95.0,29.955969,26.20,52.0,1.056209,0.0,2.038720,1126.779966,1110.0,0.991352,-0.435271
28,60.0,16.823963,0.00,120.0,15.855631,0.0,29.499148,817.308707,0.0,0.425162,-2.925484


In [ ]:
# у 1 и 5 относительно большая средняя скорость
# У 3 намного бак намного больше, чем у остальных
# у 19 очень маленькая средняя скорость 
# 28 часто стоит выключенным + с пустым баком, но потребляет много топлива
# + средний уровень топлива небольшой => наврное, на большие расстояния не ездит

In [122]:
dict_df = dict_df_vehicle_i(28)
df_sum = df_summary(dict_df, merge_how='inner')

In [138]:
print('Доля записей, в которых бак пустой:', (df_sum['fuellevel']==0).mean())
print('Доля записей, в которых зажигание выключено:', (df_sum['ingection']==0).mean())
print('Доля записей, в которых выполнено и то и то:', ((df_sum['ingection']==0) & (df_sum['fuellevel']==0)).mean())

Доля записей, в которых бак пустой: 0.593607579755337
Доля записей, в которых зажигание выключено: 0.5748380906692252
Доля записей, в которых выполнено и то и то: 0.5747781242504197


In [137]:
# Странно
# 1. Значит ли это, что при выключенном зажигании уровень топлива полагается = 0?
# 2. Почему #3 отличается от остальных?

for vehicle_id in VEHICLE_IDS:
    dict_df = dict_df_vehicle_i(vehicle_id)
    df_sum = df_summary(dict_df, merge_how='inner')
    
    meanFuelLevel = df_sum.loc[df_sum['ingection']==0, 'fuellevel'].mean()
    print(f'Средний уровень топлива у ТС #{vehicle_id} при выключенном зажигании: {meanFuelLevel:.2f}')

Средний уровень топлива у ТС #1 при выключенном зажигании: 0.00
Средний уровень топлива у ТС #3 при выключенном зажигании: 71.30
Средний уровень топлива у ТС #5 при выключенном зажигании: 0.05
Средний уровень топлива у ТС #19 при выключенном зажигании: 0.00
Средний уровень топлива у ТС #28 при выключенном зажигании: 0.00
